# Setup

In [ ]:
%reload_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount = True)

Mounted at /content/gdrive


In [ ]:
SYS_PROJECT_DIR  = '/content/gdrive/MyDrive/Colab Notebooks/musicgen'
SYS_INPUT_DIR    = '/content/gdrive/MyDrive/Colab Notebooks/musicgen/input'
SYS_MODELING_DIR = '/content/gdrive/MyDrive/Colab Notebooks/musicgen/modeling'
SYS_OUTPUT_DIR   = '/content/gdrive/MyDrive/Colab Notebooks/musicgen/output'

In [ ]:
import os
import pandas as pd
import shutil

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

# Installs

In [ ]:
!pip uninstall packaging
!pip install packaging
!pip install -q miditoolkit
!pip install -q music21

!apt install fluidsynth | grep -E "error|warning"
!apt install libfluidsynth3 | grep -E "error|warning"
!apt install fluid-soundfont-gm | grep -E "error|warning"
!pip install -q pyfluidsynth

Found existing installation: packaging 24.0
Uninstalling packaging-24.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/packaging-24.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/packaging/*
Proceed (Y/n)? Y
  Successfully uninstalled packaging-24.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.9 MB/s eta 0:00:00


Extracting templates from packages: 100%




  Preparing metadata (setup.py) ... done


# Data Preparation

## Data Exploration

### Get basic info

In [ ]:
from pathlib import Path
from miditoolkit import MidiFile

data_musician = "Roxette"

midi_path = Path(SYS_INPUT_DIR, data_musician, "Crash! Boom! Bang!.1.mid")
midi_file = MidiFile(midi_path)

print(midi_file)

ticks per beat: 192
max tick: 68737
tempo changes: 1
time sig: 1
key sig: 1
markers: 1
lyrics: False
instruments: 11


### Play file

In [ ]:
from pathlib import Path
from music21 import midi
import fluidsynth
from IPython.display import Audio

data_musician = "Roxette"
midi_file = "Crash! Boom! Bang!.1.mid"
midi_path = Path(SYS_INPUT_DIR, data_musician, midi_file)

import re, subprocess

def convert_mid_audio(sf2, midi_file, out_dir, out_type='wav', txt_file=None, append=True):
    """
    Convert a single midi file to an audio file.

    Args:
        sf2 (str):        the file path for a .sf2 soundfont file
        midi_file (str):  the file path for the .mid midi file to convert
        out_dir (str):    the directory path for where to write the audio out
        out_type (str):   the output audio type (see 'fluidsynth -T help' for options)
    """
    fbase = os.path.splitext(os.path.basename(midi_file))[0]
    out_file = out_dir + '/' + fbase + '.' + out_type

    subprocess.call(['fluidsynth', '-T', out_type, '-F', out_file, '-ni', sf2, midi_file])

wav_dir =  Path(SYS_INPUT_DIR, 'exploration', data_musician)
convert_mid_audio('/usr/share/sounds/sf2/FluidR3_GM.sf2', midi_path, wav_dir)

In [ ]:
from pathlib import Path
from scipy.io import wavfile

data_musician = "Roxette"
wav_dir =  Path(SYS_INPUT_DIR, 'exploration', data_musician)
wav_file = "Crash! Boom! Bang!.1.wav"
wav_path = Path(wav_dir, wav_file)

# from IPython.display import Audio, display

# display(Audio(wav_path, rate=44100))

# from IPython.display import Audio
# data = wavfile.read(str(wav_path))
# # Audio(wav_path, rate=44100)

import IPython.display as ipd
ipd.Audio(wav_path) # load a local WAV file

In [ ]:
from IPython.display import HTML

def play_wav_in_jupyter(file_path):
    audio_html = f"""
    <audio controls>
      <source src="{file_path}" type="audio/wav">
      Your browser does not support the audio element.
    </audio>
    """
    display(HTML(audio_html))

data_musician = "Roxette"
wav_dir =  Path(SYS_INPUT_DIR, 'exploration', data_musician)
wav_file = "Crash! Boom! Bang!.1.wav"
wav_path = Path(wav_dir, wav_file)


# Call the function to display the audio player
play_wav_in_jupyter(str(wav_path))

<audio controls=true src="attachment:tuba.wav"/>

<audio controls=true src="tuba.wav"/>

In [ ]:
data

(44100,
 array([[-1, -1],
        [-1, -1],
        [ 0, -1],
        ...,
        [-1, -1],
        [-1, -1],
        [-1, -1]], dtype=int16))

## Chunking the Dataset

In [ ]:
MAX_NB_BAR = 8
MIN_NB_NOTES = 20

In [ ]:
from pathlib import Path
from tqdm import tqdm
from copy import deepcopy
from miditoolkit import MidiFile
from math import ceil

data_musician = "Roxette"

data_problem_files = [
    Path(SYS_INPUT_DIR, data_musician, "It Must Have Been Love (live studio).3.mid"),
    Path(SYS_INPUT_DIR, data_musician, "The Look.mid"),
]
merged_out_dir = Path(SYS_INPUT_DIR, data_musician, f"{data_musician}-chunked")
merged_out_dir.mkdir(parents=True, exist_ok=True)
midi_paths = list(Path(SYS_INPUT_DIR, data_musician).glob("**/*.mid"))

for i, midi_path in enumerate(tqdm(midi_paths, desc="Chunkiing MID Audio file")):
    try:
        # Determine the output directory for this file
        relative_path = midi_path.relative_to(Path(SYS_INPUT_DIR, data_musician))
        output_dir = merged_out_dir / relative_path.parent
        output_dir.mkdir(parents=True, exist_ok=True)

        # Check if chunks already exist
        chunk_paths = list(output_dir.glob(f"{midi_path.stem}_*.mid"))
        if len(chunk_paths) > 0:
            print(f"Chunks for {midi_path} already exist, skipping...")
            continue

        # Loads MIDI, merges and saves it
        midi = MidiFile(midi_path)
        ticks_per_cut = MAX_NB_BAR * midi.ticks_per_beat * 4
        nb_cuts = ceil(midi.max_tick / ticks_per_cut)
        if nb_cuts < 2:
            continue

        if midi_path in data_problem_files:
              print(f"Skipping problematic MIDI file {midi_path}")
              continue

        print(f"Processing {midi_path}")
        midis = [deepcopy(midi) for _ in range(nb_cuts)]

        for j, track in enumerate(midi.instruments):  # sort notes as they are not always sorted right
            track.notes.sort(key=lambda x: x.start)
            for midi_short in midis:  # clears notes from shorten MIDIs
                midi_short.instruments[j].notes = []
            for note in track.notes:
                cut_id = note.start // ticks_per_cut
                note_copy = deepcopy(note)
                note_copy.start -= cut_id * ticks_per_cut
                note_copy.end -= cut_id * ticks_per_cut
                midis[cut_id].instruments[j].notes.append(note_copy)

        # Saving MIDIs
        for j, midi_short in enumerate(midis):
            if sum(len(track.notes) for track in midi_short.instruments) < MIN_NB_NOTES:
                continue
            midi_short.dump(output_dir / f"{midi_path.stem}_{j}.mid")

    except Exception as e:
        print(f"An error occurred while processing {midi_path}: {e}")

CHUNKING MIDIS:   0%|          | 0/117 [00:00<?, ?it/s]

Processing /content/gdrive/MyDrive/Colab Notebooks/musicgen/input/Roxette/It Must Have Been Love (live studio).mid


CHUNKING MIDIS:   1%|          | 1/117 [00:01<03:46,  1.95s/it]

Processing /content/gdrive/MyDrive/Colab Notebooks/musicgen/input/Roxette/Fading Like a Flower (Every Time You Leave).mid


CHUNKING MIDIS:   2%|▏         | 2/117 [00:05<05:14,  2.73s/it]

...

CHUNKING MIDIS:  98%|█████████▊| 115/117 [03:22<00:05,  2.57s/it]

Processing /content/gdrive/MyDrive/Colab Notebooks/musicgen/input/Roxette/The Look.9.mid


CHUNKING MIDIS:  99%|█████████▉| 116/117 [03:25<00:02,  2.66s/it]

Processing /content/gdrive/MyDrive/Colab Notebooks/musicgen/input/Roxette/Dangerous (7_ version).mid


CHUNKING MIDIS: 100%|██████████| 117/117 [03:26<00:00,  1.77s/it]


# Modeling

# Sleep

In [ ]:
import time
time.sleep(3600)